# Data Generation

In [32]:
import random
import time
import tracemalloc
import matplotlib.pyplot as plt
import statistics
import math

def generate_synthetic_dna(alphabet=['A', 'T', 'C', 'G'], length=50, distribution=None, seed=None):

    if distribution is None:
        distribution = [1.0 / len(alphabet)] * len(alphabet)

    if seed is not None:
        random.seed(seed)

    sequence = ''.join(random.choices(alphabet, weights=distribution, k=length))

    return sequence

In [23]:
generate_synthetic_dna(distribution = [0.25, 0.4, 0.1, 0.25])

'TCATTTGCTTGGTATATTATGTATCTAAGAACTGTAATGTTTATTTAGTT'

# Scoring & Algorithms

In [50]:
def score(a, b):
    if a == b:
        return 1
    elif a == '-' or b == '-':
        return -1
    else:
        return -1

def nw_score_prefix(x, y):

    p = [j * -1 for j in range(len(y) + 1)]

    for r in range(1, len(x) + 1):

        q = [0] * (len(y) + 1)
        q[0] = -r

        for c in range(1, len(y) + 1):

            mm = p[c-1] + score(x[r-1], y[c-1])

            dl = p[c] + score(x[r-1], '-')

            ins = q[c-1] + score('-', y[c-1])

            q[c] = max(mm, dl, ins)

        p = q
    return p

def nw_score_suffix(x, y):

    rx, ry = x[::-1], y[::-1]

    rs = nw_score_prefix(rx, ry)

    return rs

def hirschberg(x, y, w):

    m, n = len(x), len(y)

    if m == 0:
        to_return_m = ('-' * n, y, sum(score('-', ch) for ch in y))
        return to_return_m

    elif n == 0:
        to_return_n = (x, '-' * m, sum(score(ch, '-') for ch in x))
        return to_return_n

    elif m == 1 or n == 1:
        sc, aln = global_align(x, y)     # global_align returns (score, alignment_string)
        a1, a2 = aln.split('\n')
        return (a1, a2, sc)

    else:
        md = m // 2

        sl = nw_score_prefix(x[:md], y)
        sr = nw_score_suffix(x[md:], y)

        mx = None
        pk = 0

        for k in range(n+1):
            val = sl[k] + sr[n-k]
            if mx is None or val > mx:
                mx = val
                pk = k

        # store steps in w
        w.append({
            'mid': md,
            'part': pk,
            'val': mx
        })

        a1, a2, sc1 = hirschberg(x[:md], y[:pk], w)
        b1, b2, sc2 = hirschberg(x[md:], y[pk:], w)

        return (a1 + b1, a2 + b2, sc1 + sc2)

In [52]:
UP = (-1, 0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)


def traceback_global(v, w, pointers):

    i, j = len(v), len(w)
    new_v = []
    new_w = []
    while True:
        di, dj = pointers[i][j]
        if (di, dj) == LEFT:
            new_v.append('-')
            new_w.append(w[j-1])
        elif (di, dj) == UP:
            new_v.append(v[i-1])
            new_w.append('-')
        elif (di, dj) == TOPLEFT:
            new_v.append(v[i-1])
            new_w.append(w[j-1])

        i, j = i + di, j + dj
        if i <= 0 and j <= 0:
            break

    return ''.join(new_v[::-1]) + '\n' + ''.join(new_w[::-1])

def global_align(v, w):

    M = [[0 for _ in range(len(w)+1)] for _ in range(len(v)+1)]
    pointers = [[ORIGIN for _ in range(len(w)+1)] for _ in range(len(v)+1)]

    for i in range(1, len(v)+1):
        M[i][0] = M[i-1][0] + score(v[i-1], '-')
        pointers[i][0] = UP

    for j in range(1, len(w)+1):
        M[0][j] = M[0][j-1] + score('-', w[j-1])
        pointers[0][j] = LEFT

    for i in range(1, len(v)+1):
        for j in range(1, len(w)+1):
            up_score    = M[i-1][j]   + score(v[i-1], '-')
            left_score  = M[i][j-1]   + score('-', w[j-1])
            diag_score  = M[i-1][j-1] + score(v[i-1], w[j-1])

            # Choose the max
            max_score = max(up_score, left_score, diag_score)

            M[i][j] = max_score
            if max_score == up_score:
                pointers[i][j] = UP
            elif max_score == left_score:
                pointers[i][j] = LEFT
            else:
                pointers[i][j] = TOPLEFT

    final_score = M[len(v)][len(w)]
    alignment = traceback_global(v, w, pointers)
    return final_score, alignment



In [51]:
if __name__ == "__main__":

    # Examples sequences
    seq1 = "ACGTACGTAC"
    seq2 = "TACGTCGT"

    intermediate_data = []
    aln1, aln2, aln_score = hirschberg(seq1, seq2, intermediate_data)
    print("Alignment 1:", aln1)
    print("Alignment 2:", aln2)
    print("Score:", aln_score)
    print("Intermediate points:", intermediate_data)

Alignment 1: -ACGTACGTAC
Alignment 2: TACGT-CGT--
Score: 3
Intermediate points: [{'mid': 5, 'part': 5, 'val': 3}, {'mid': 2, 'part': 3, 'val': 2}, {'mid': 1, 'part': 2, 'val': 1}, {'mid': 1, 'part': 1, 'val': 1}, {'mid': 2, 'part': 2, 'val': 1}, {'mid': 1, 'part': 1, 'val': 2}]


# Benchmarking

In [ ]:
lengths = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600]
num_samples = 30

nw_time_means = []
nw_time_vars = []
nw_mem_means = []
nw_mem_vars = []

hir_time_means = []
hir_time_vars = []
hir_mem_means = []
hir_mem_vars = []

all_nw_times = {}
all_nw_mems = {}
all_hir_times = {}
all_hir_mems = {}

for L in lengths:
    nw_times = []
    nw_mems = []
    hir_times = []
    hir_mems = []

    for i in range(num_samples):
        seq1 = generate_synthetic_dna(length=L)
        seq2 = generate_synthetic_dna(length=L)

        tracemalloc.start()
        start = time.perf_counter()
        i = global_align(seq1, seq2)
        end = time.perf_counter()
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()

        nw_times.append(end - start)
        nw_mems.append(peak)

        # Hirschberg timing and memory
        tracemalloc.start()
        start = time.perf_counter()
        i = hirschberg(seq1, seq2, [])
        end = time.perf_counter()
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()

        hir_times.append(end - start)
        hir_mems.append(peak)

    all_nw_times[L] = nw_times
    all_nw_mems[L] = nw_mems
    all_hir_times[L] = hir_times
    all_hir_mems[L] = hir_mems

    nw_time_mean = statistics.mean(nw_times)
    nw_time_var = statistics.pvariance(nw_times)
    nw_mem_mean = statistics.mean(nw_mems)
    nw_mem_var = statistics.pvariance(nw_mems)

    nw_time_means.append(nw_time_mean)
    nw_time_vars.append(nw_time_var)
    nw_mem_means.append(nw_mem_mean)
    nw_mem_vars.append(nw_mem_var)

    hir_time_mean = statistics.mean(hir_times)
    hir_time_var = statistics.pvariance(hir_times)
    hir_mem_mean = statistics.mean(hir_mems)
    hir_mem_var = statistics.pvariance(hir_mems)

    hir_time_means.append(hir_time_mean)
    hir_time_vars.append(hir_time_var)
    hir_mem_means.append(hir_mem_mean)
    hir_mem_vars.append(hir_mem_var)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8,8))

nw_time_std = [math.sqrt(v) for v in nw_time_vars]
hir_time_std = [math.sqrt(v) for v in hir_time_vars]
nw_mem_std = [math.sqrt(v) for v in nw_mem_vars]
hir_mem_std = [math.sqrt(v) for v in hir_mem_vars]

ax1.errorbar(lengths, nw_time_means, yerr=nw_time_std, marker='o', label='Needleman-Wunsch')
ax1.errorbar(lengths, hir_time_means, yerr=hir_time_std, marker='o', label='Hirschberg')
ax1.set_xlabel('Sequence length')
ax1.set_ylabel('Time (s)')
ax1.set_title('Time comparison (mean ± std dev)')
ax1.legend()

ax2.errorbar(lengths, nw_mem_means, yerr=nw_mem_std, marker='o', label='Needleman-Wunsch')
ax2.errorbar(lengths, hir_mem_means, yerr=hir_mem_std, marker='o', label='Hirschberg')
ax2.set_xlabel('Sequence length')
ax2.set_ylabel('Peak memory (bytes)')
ax2.set_title('Memory comparison (mean ± std dev)')
ax2.legend()

plt.tight_layout()
plt.show()

print("Needleman-Wunsch Time Means:", nw_time_means)
print("Needleman-Wunsch Time Variances:", nw_time_vars)
print("Hirschberg Time Means:", hir_time_means)
print("Hirschberg Time Variances:", hir_time_vars)

print("Needleman-Wunsch Memory Means:", nw_mem_means)
print("Needleman-Wunsch Memory Variances:", nw_mem_vars)
print("Hirschberg Memory Means:", hir_mem_means)
print("Hirschberg Memory Variances:", hir_mem_vars)

# Print all individual measurements
#print("All NW Times:", all_nw_times)
#print("All NW Mem:", all_nw_mems)
#print("All Hir Times:", all_hir_times)
#print("All Hir Mem:", all_hir_mems)

In [36]:
nw_times

[2.8490217729995493,
 1.9405471139998554,
 1.9015571789987007,
 1.9398542100007035,
 1.920376045998637,
 2.804286997001327,
 1.922897627000566,
 1.913336753999829,
 1.8909613849991729,
 1.9086576720001176,
 2.788167300999703,
 1.9270375220003189,
 1.9106215690007957,
 1.9173417460006021,
 1.912267129000611,
 2.858783386000141,
 1.923011734001193,
 2.011529753999639,
 1.9206159440000192,
 1.9473676220004563,
 3.0118849129994487,
 1.9137148469999374,
 1.9479146060002677,
 1.9264727379995747,
 1.9447280839995074,
 3.0946570899996004,
 1.927253402000133,
 1.9364976019987807,
 1.8316563530006533,
 1.932971012998678]

In [45]:
all_nw_mems

{50: [73356,
  65704,
  57590,
  63690,
  64278,
  59894,
  58818,
  63050,
  62882,
  65486,
  60608,
  72124,
  61196,
  58664,
  58608,
  61448,
  61734,
  62050,
  60294,
  78614,
  66296,
  61590,
  68648,
  66630,
  72368,
  62670,
  68620,
  67762,
  60966,
  63110],
 100: [275132,
  253376,
  268746,
  291742,
  263120,
  256589,
  308136,
  298424,
  291812,
  296268,
  282718,
  287548,
  285297,
  326878,
  282416,
  308244,
  254928,
  277630,
  302202,
  244610,
  301405,
  286580,
  268286,
  263299,
  278322,
  267006,
  238354,
  307582,
  254164,
  294502],
 150: [671398,
  644252,
  677084,
  628314,
  675678,
  652041,
  588942,
  611030,
  646784,
  594658,
  629708,
  619650,
  614134,
  612670,
  586896,
  612268,
  667358,
  669741,
  607866,
  612300,
  605982,
  617046,
  710005,
  602914,
  628582,
  608382,
  630750,
  643984,
  622028,
  684882],
 200: [1150860,
  1151100,
  1014924,
  1028724,
  1059532,
  1278013,
  1095390,
  1161634,
  1069460,
  1033874